Begining of analysis

 

In [ ]:
#import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import date, datetime
import os, sys 
import io
import ee


In [ ]:
## set up pandas dataframe using set_options to warn when we are working on a copy instead of ooriginal dataframe
pd.set_option('mode.chained_assignment','warn')

In [ ]:
# dataframe will display without being truncated
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_columns', 25)
pd.set_option('display.width', 768)

In [ ]:
#mounting the google drive to access the files
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#import csv that was cleaned up in cat_desc_normalize.ipynb
csv_path = "/content/drive/My Drive/p_cat_clean.csv"


In [ ]:
#list of columns
cols=list(pd.read_csv(csv_path, nrows=1))
cols


In [ ]:
# backup of clean dataframe, types will not be fully preserved so on import reassign, but easier with unconformity removed
df = pd.read_csv(csv_path, dtype={'Jobs Saved':int, 'NAICS Code': str, 'NAICS Category': str, 'Zip': str, }, parse_dates=[10] )
df.sample()

In [ ]:
#found some weird characters
df['Bank']=df['Bank'].str.lstrip('\t')

In [ ]:
#types are correct
df.dtypes

In [ ]:
df.shape

In [ ]:
nat_loan_stats=df['Loan Amount'].describe()
nat_loan_stats

In [ ]:
national_loan_avg=df['Loan Amount'].mean()
national_loan_avg

In [ ]:
national_loan_med=df['Loan Amount'].median()
national_loan_med

In [ ]:
national_loan_sum=df['Loan Amount'].sum()
national_loan_sum

In [ ]:
national_loan_count=df['Loan Amount'].count()

In [ ]:
national_loan_sum=df['Jobs Saved'].sum()
national_loan_sum

In [ ]:
nat_pct_dif=national_loan_avg/national_loan_med * 100
nat_pct_dif

In [ ]:
#summary table of stats
nat_summary = pd.DataFrame({
    'Loan Avg': national_loan_avg,
    'Loan Median': national_loan_med,
    'Loan Sum': national_loan_sum,
    'Loan Count': national_loan_count,
    'Jobs Saved': national_loan_sum,
    'Avg over Median': nat_pct_dif,
},index=[0])
nat_summary.head(3)

# Summary Table

Loan Avg : $43,003.12 | Median Loan : $30,000 | Loan Sum: $10,491,212,067| Loan Count: 243,964 | Jobs Saved: 1,505,064 | Avg loan amount over Median: 143.34%


In [ ]:
nat_summary['Loan Avg'] = nat_summary.loc[:,'Loan Avg'].map("${:,.2f}".format)
nat_summary['Loan Median'] = nat_summary.loc[:,'Loan Median'].map("${:,.2f}".format)
nat_summary['Loan Sum'] = nat_summary.loc[:,'Loan Sum'].map("{:,}".format)
nat_summary['Loan Count'] = nat_summary.loc[:,'Loan Count'].map("{:,}".format)
nat_summary['Jobs Saved'] = nat_summary.loc[:,'Jobs Saved'].map("{:,}".format)
nat_summary['Avg over Median'] = nat_summary.loc[:,'Avg over Median'].map("{:,.2f}%".format)
nat_summary






In [ ]:
#who are the top lenders by amount? Creating a summary table that can be exported to csv for analysis in SQL/ Tableau later
top_bank_sums=df.groupby('Bank').sum()['Loan Amount']
top_bank_sums

In [ ]:
top_bank_median=df.groupby('Bank').median()['Loan Amount']
top_bank_median

In [ ]:
top_bank_average=df.groupby('Bank').mean()['Loan Amount']
top_bank_average

In [ ]:
banks_summary_df=pd.DataFrame({
    'Loan Sum' : top_bank_sums,
    'Average Loan' : top_bank_average,
    'Median Loan' : top_bank_median
  
})
banks_summary_df.head()


In [ ]:
banks_summary_df['Loan Sum'] = banks_summary_df.loc[:,'Loan Sum'].map("${:,.2f}".format)
banks_summary_df['Average Loan'] = banks_summary_df.loc[:,'Average Loan'].map("${:,.2f}".format)
banks_summary_df['Median Loan'] = banks_summary_df.loc[:,'Median Loan'].map("${:,.2f}".format)
banks_summary_df.sample(30)


In [ ]:
result=8142/243964
result

In [ ]:
#some of the top institutions by state that considering they are regional have a relatively large percentage of the loans
# for example OH - Huntington bank did 3.3% of the loan count for this dataset Mississippi
top_bks_state=df.groupby('State')['Bank'].value_counts().nlargest(15)
top_bks_state

In [ ]:
category_summary_df.to_csv('bks_by_state.csv', index=0)
!cp bks_by_state.csv "drive/My Drive/"

In [ ]:
loans_cat=df.groupby('description')['Loan Amount'].sum()
loans_cat.head()

In [ ]:
jobs_cat=df.groupby('description')['Jobs Saved'].sum()
jobs_cat.head()

In [ ]:
median_cat=df.groupby('description')['Loan Amount'].median()
median_cat.head()

In [ ]:
mean_cat=df.groupby('description')['Loan Amount'].mean()
mean_cat.head()

In [ ]:
category_summary_df=pd.DataFrame({
    'Jobs Saved' : jobs_cat,
    'Median Loan' : median_cat,
    'Average Loan' : mean_cat,
    'Total Loans': loans_cat,
  
})
category_summary_df.head()

No surprise - Hospitality industry seemed to take the biggest hit with Construction and Health Care loans the next highest number of jobs saved. Seemingly unexpected are Professional and Technical Services and Waste Management which may deserve a "look under the hood".

In [ ]:
category_summary_df['Jobs Saved'] = category_summary_df.loc[:,'Jobs Saved'].map("{:,}".format)
category_summary_df['Median Loan'] = category_summary_df.loc[:,'Median Loan'].map("${:,.2f}".format)
category_summary_df['Average Loan'] = category_summary_df.loc[:,'Average Loan'].map("${:,.2f}".format)
category_summary_df['Total Loans'] = category_summary_df.loc[:,'Total Loans'].map("${:,.2f}".format)
category_summary_df.head(30)

In [ ]:
#save to csv and will convert to a table in SQL / Tableau for further analysis 

category_summary_df.to_csv('category_summary.csv', index=0)
!cp category_summary.csv "drive/My Drive/"

In [ ]:
sum_by_state=df.groupby('State',)['Loan Amount'].sum() 
sum_by_state.nlargest(8)

In [ ]:
# df['Loan PCL']=pd.qcut(df['Loan Amount'], q=10)

bin_labels_10= ['10%', '20%', '30%', '40%', '50%', '60%', '70%', '80%', '90%' ]
df['Loan Percentile'] = pd.qcut(df['Loan Amount'],
                              q=[0,  .10, .20, .30, .40, .50, .60, .70, .80, 1],
                              labels=bin_labels_10)
df.head()

In [ ]:
bin_labels_4 = ['25%', '45%', '50%', '75%', ]
df['Loan Quartile'] = pd.qcut(df['Loan Amount'],
                              q=[0, .25, .50, .75, 1],
                              labels=bin_labels_4)
df.head()

The 90th percentile of loans account for 20% of total in terms of loan amount


In [ ]:
df['Loan Quartile'].value_counts()

In [ ]:
df.sample()

In [ ]:
loan_qt = np.percentile(df['Loan Amount'], [25, 50, 75, 100])
loan_qt


In [ ]:
high_pct = df[(df["Loan Percentile"] == '90%')]

high_pct.head(1)


In [ ]:
#save to csv and will convert to a table in SQL / Tableau for further analysis 

high_pct.to_csv('top_percentile_banks.csv', index=0)
!cp top_percentile_banks.csv "drive/My Drive/"

Some lines of inquiry to pursue pertaining to race and gender
From the subset of people who self identified racially, Asians borrowed a disproporianatly larger amount as a percentage of population and Hispanics less. (Asians are less than 6% and Hispanics 16%). Puerto Ricans are not necessarily accounted for in the Hispanic statistics. It is hard to say whether or not this is a true representation of percentages of all loans because this dataset is small (less than 5% of the total) and only reflects the people who were willing to answer the race inquiry. Asians and Whites had roughly the same gender percentages, median and average loan amounts. Hispanics and Blacks seemed to incur lower levels of debt.

In [ ]:
state_race_cts=df.groupby('State',)['Race'].value_counts()
state_race_cts.head()

In [ ]:
df

In [ ]:

race_avg=df.groupby('Race')['Loan Amount'].mean()
race_avg

In [ ]:
# Define plot space
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title = "Average Loan Amount by Race",
       xlabel = "", 
       ylabel = "",

       )
labels=race_qt.index
explode = (0.0, 0.075, 0.0, 0.0, 0, 0,  0, 0)
# Define x and y axes
ax.pie(race_qt, explode=explode, labels=labels)


In [ ]:
# df.groupby('Race', 'Loan Amount').quantile()
race_count=df.groupby('Race').count()
race_count

In [ ]:
# Define plot space
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title = "",
       xlabel = "", 
       ylabel = "",

       )
labels=race_count.index
explode = (0.0, 0.0, 0.0, 0.0, 0., 0.0, 0.0, 0.05)
left  = 0.125  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.5  # the amount of width reserved for blank space between subplots
hspace = 0.5   # the amount of height reserved for white space between subplots
# These two can be called on 'fig' instead of 'plt' too
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top,
                wspace=wspace, hspace=hspace)

# Define x and y axes
ax.pie(race_count['Loan Amount'], explode=explode, labels=labels, rotatelabels=True)
plt.savefig('race_countof_loans.png')

In [ ]:
race_qt.head()

In [ ]:
vt_gen_qt=df.groupby(['Race', 'Veteran'])['Loan Amount'].quantile()
vt_gen_qt

In [ ]:
vt_cat=df.groupby(['Race', 'Veteran'])['description'].value_counts()
vt_cat

In [ ]:
df.sample()

In [ ]:
rc_jbs=df.groupby(['Race', 'Gender'])['Jobs Saved'].sum()
rc_jbs

In [ ]:
race_median=df.groupby(['Race', 'Gender'])['Loan Amount'].median()
race_median

In [ ]:
race_mean=df.groupby(['Race', 'Gender'])['Loan Amount'].mean()
race_mean

In [ ]:
df.shape

In [ ]:
rc_gen_qt=df.groupby(['Race', 'Gender'])['Loan Amount'].quantile()


In [ ]:
counts_by_race=df.groupby(['Race', 'Gender'])['Loan Amount'].count()
counts_by_race

In [ ]:
race_summary= pd.DataFrame({
    'Count' : counts_by_race,
    'Quantile' : rc_gen_qt,
    'Mean' : race_mean,
    'Median' : race_median,
    'Jobs' : rc_jbs


})
race_summary.head()

In [ ]:
race_summary['Count'] = race_summary.loc[:,'Count'].map("{:,}".format)
race_summary['Jobs'] = race_summary.loc[:,'Jobs'].map("{:,}".format)
race_summary['Quantile'] = race_summary.loc[:,'Quantile'].map("${:,.2f}".format)
race_summary['Mean'] = race_summary.loc[:,'Mean'].map("${:,.2f}".format)
race_summary['Median'] = race_summary.loc[:,'Median'].map("${:,.2f}".format)
race_summary

In [ ]:
#save to csv and will convert to a table in SQL / Tableau for further analysis 

race_summary.to_csv('race_summary.csv', index=0)
!cp race_summary.csv "drive/My Drive/"

In [ ]:
df.sample()


In [ ]:
cat_sum=df.groupby('description')['Jobs Saved'].sum()
cat_sum.head()

In [ ]:
cat_sum.sort_values().plot(kind='barh', y='Jobs Saved', subplots=True, figsize=(12,9), color="lightskyblue")

plt.savefig('category_jobs_saved.png', bbox_inches="tight")

In [ ]:
cat_loan=df.groupby('description')['Loan Amount'].sum()
cat_loan.head()

In [ ]:
cat_loan.sort_values().plot(kind='barh', y='Jobs Saved', subplots=True, figsize=(12,9), color="lightcoral")

plt.savefig('catgory_loans.png', bbox_inches="tight")

In [ ]:
count_race = df.groupby('Race').agg('count')
count_desc = df.groupby('description').agg('count')
count_sum = len(df)




In [ ]:
# def group_lower_ranking_values(column):
#     pie_counts = df.groupby(column).agg('count')
#     pct_value = df[lambda df: df.columns[0]].quantile(.75)
#     values_below_pct_value = df[lambda df: df.columns[0]].loc[lambda s: s < pct_value].index.values
#     def fix_values(row):
#         if row[column] in values_below_pct_value:
#             row[column] = 'Other'
#         return row 
#     pie_grouped = df.apply(fix_values, axis=1).groupby(column).agg('count')
#     return pie_grouped

# racial_index = group_lower_ranking_values('Race')
# description_index = group_lower_ranking_values('description')

In [ ]:
racial_index=df.groupby('Race').agg('count')

In [ ]:
gender_loan_amount=df.groupby('Gender')['Loan Amount'].agg('sum')

In [ ]:
%matplotlib inline

In [ ]:
# Define plot space
fig, ax = plt.subplots(figsize=(10, 6))
ax.set(title = "Total Percent of Loans by Gender",
       xlabel = "", 
       ylabel = "",

       )
labels=gender_loan_amount.index
explode = (0.0, 0.05)
left  = 0.125  # the left side of the subplots of the figure
right = 0.9    # the right side of the subplots of the figure
bottom = 0.1   # the bottom of the subplots of the figure
top = 0.9      # the top of the subplots of the figure
wspace = 0.5  # the amount of width reserved for blank space between subplots
hspace = 0.5   # the amount of height reserved for white space between subplots
# These two can be called on 'fig' instead of 'plt' too
plt.subplots_adjust(left=left, bottom=bottom, right=right, top=top,
                wspace=wspace, hspace=hspace)

# Define x and y axes
ax.pie(gender_loan_amount, explode=explode, labels=labels, rotatelabels=False)
plt.savefig('gender_loan_total.png', bbox_inches='tight')

In [ ]:
df.to_csv('ppl_ppp_w_quantiles_and_categories.csv', index=0)
!cp people_ppp_super.csv "drive/My Drive/"
